In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils, Networks

In [3]:
import numpy as np
import tensorflow as tf
import os
from os import path
from utils import load_embeddings_and_ids, concatenate_featmats
from sklearn.preprocessing import StandardScaler
from Networks import ContentBasedLearn2RankNetwork_Precomputation

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
resnet50 = load_embeddings_and_ids('/mnt/workspace/Ugallery/ResNet50/', 'flatten_1.npy', 'ids')

In [6]:
alexnet = load_embeddings_and_ids('/mnt/workspace/Ugallery/AlexNet/', 'fc7.npy', 'ids.npy')

In [7]:
inceptionv3 = load_embeddings_and_ids('/mnt/workspace/Ugallery/InceptionV3/', 'avg_pool.npy', 'ids')

In [8]:
embedding_list = [resnet50, alexnet, inceptionv3]

In [9]:
artwork_ids_set = set()
for embedding in embedding_list:
    artwork_ids_set.update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

13297

In [10]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [11]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [12]:
concat_featmat.shape

(13297, 8192)

In [13]:
MODEL_PATH = ('/mnt/workspace/pamessina_models/ugallery/youtube_like/v16(10M-400K,avg+max,rsnt50+alxnt+incepv3,u(300,300,200)i(200,200),+p(rl&fk)-np,+npfavc(rl&fk)-nfavc,+nxt+hdaccklst-nfavc,fg.6,vcf.1/')

In [14]:
with tf.Graph().as_default():
    network = ContentBasedLearn2RankNetwork_Precomputation(
        pretrained_embedding_dim=concat_featmat.shape[1],
        item_layer_units=[200,200],
    )
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        item_vectors = network.precompute_tensors(sess, concat_featmat)

INFO:tensorflow:Restoring parameters from /mnt/workspace/pamessina_models/ugallery/youtube_like/v16(10M-400K,avg+max,rsnt50+alxnt+incepv3,u(300,300,200)i(200,200),+p(rl&fk)-np,+npfavc(rl&fk)-nfavc,+nxt+hdaccklst-nfavc,fg.6,vcf.1/


In [15]:
item_vectors.shape

(13297, 200)

In [16]:
dir_path = ('/mnt/workspace/Ugallery/Youtube-like/v16(10M-400K,avg+max,rsnt50+alxnt+incepv3,u(300,300,200)i(200,200),+p(rl&fk)-np,+npfavc(rl&fk)-nfavc,+nxt+hdaccklst-nfavc,fg.6,vcf.1/')
os.makedirs(dir_path, exist_ok=True)
item_vectors.dump(dir_path + "item_vectors.npy")
with open(dir_path + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)